In [1]:
!pip install Unidecode --quiet
import unidecode 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.7 MB/s eta 0:00:00


In [2]:
import numpy as np 
import re
import nltk
from nltk.corpus import stopwords

# Se cargan los corpus


In [3]:
# Montamos el drive
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ANXIETY/corpus_anxiety.txt', 'r') as f:
    corpus_anxiety = f.read()

In [8]:
# Visualizamos el contenido del corpus de ansiedad
print(corpus_anxiety[:838])

Extreme anxiety after Road Rage incident Almost 3 months ago a lady had crazy road rage at me for going the speed limit she passes me and starts taking pictures of my car, follows me and then films me claiming I tried to run her off the road and she is calling the cops. I have been a wreck ever since, I feel every day like today the cops are going to come arrest me because of what this lady said. I dont know how to get over this.
Cant sleep because I cant breathe I am currently sick to where I cant breathe until I cough the mucus out of my windpipe. I feel alright and then the next minute I am choking. Tonight I had my first panic attack because I thought I wasnt going to be able to breathe anymore. Now I am laying in bed and it feels like I manually have to do each breath and I cant sleep.
Is this anxiety? I need help. Hello.


## Separamos por documentos

In [12]:
def separate_documents(corpus):
    # Inicialización de variables
    documents = []
    documents = corpus.strip().split('\n')

    return documents

In [13]:
doc = separate_documents(corpus_anxiety)

In [17]:
doc[89]

'ive been prescribed Ativan for emergency situations, but ive been taking them almost every day this month. ive been so anxious its been unreal. my living situation is up in the air i was supposed to move in with my boyf but all of a sudden were not getting along, i quit my job cuz they took uber advantage of me, and ive been isolating myself a lot as a result of all of my panicky-ness lol. i do also feel optimistic-- i have two job opportunities lined up just waiting for me to say yes, i have another good housing option lined up, etc etc. but i just feel so anxious sometimes that i paralyze myself, and by that point im so worked up that i need an Ativan so i dont feel like imma have a heart attack. fuck.'

In [18]:
# Dividimos los documentos largos en fragmentos más pequeños, debido al limite de GPT-2 con los tokens
max_tokens_per_document = 188
fragmented_documents = []

for document in doc:
    tokens = document.split()  # Dividir el documento en tokens individuales
    fragments = []

    while tokens:
        fragment = tokens[:max_tokens_per_document]  # Tomar los primeros max_tokens_per_document tokens
        tokens = tokens[max_tokens_per_document:]  # Eliminar los tokens ya tomados del documento original
        fragmented_document = ' '.join(fragment)  # Unir los tokens en un nuevo fragmento de documento
        fragments.append(fragmented_document)

    fragmented_documents.extend(fragments)

doc = fragmented_documents

# Creación del script del dataset

In [19]:
# Se crea un script de dataset utilizando la librería de Hugging Face
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.7 MB/s eta 0:00:00


In [20]:
from datasets import Dataset, DatasetDict

In [21]:
# Se crea una lista de diccionarios con los campos "id" y "text" para cada documento en doc
data = [{"id": str(i), "text": doc[i]} for i in range(len(doc))]

In [22]:
# Se crea el dataset con la lista de diccionarios, hacemos el split
train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))
test_size = len(data) - train_size - val_size

train_data = data[:train_size]
val_data = data[train_size:train_size+val_size]
test_data = data[train_size+val_size:]

dataset = DatasetDict({'train': train_data, 'validation': val_data, 'test': test_data})

In [23]:
# Obtener los conjuntos de entrenamiento del objeto dataset
train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

In [24]:
import os

In [25]:
# Rutas completas donde se almacenarán los archivos de texto de los conjuntos
train_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ANXIETY/split', 'train.txt')
val_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ANXIETY/split', 'validation.txt')
test_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ANXIETY/split', 'test.txt')

In [26]:
# Abrir el archivo de texto para escribir los datos de entrenamiento
with open(train_file_path, 'w', encoding='utf-8') as f:
    for example in train_data:
        # Escribir cada ejemplo de entrenamiento en una línea separada
        f.write(example['text'] + '\n')

# Abrir el archivo de texto para escribir los datos de validacion
with open(val_file_path, 'w', encoding='utf-8') as f:
    for example in val_data:
        # Escribir cada ejemplo de validacion en una línea separada
        f.write(example['text'] + '\n')

# Abrir el archivo de texto para escribir los datos de test
with open(test_file_path, 'w', encoding='utf-8') as f:
    for example in test_data:
        # Escribir cada ejemplo de test en una línea separada
        f.write(example['text'] + '\n')

#Entrenamiento del modelo

In [27]:
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.1 MB/s eta 0:00:00


In [28]:
!git clone https://github.com/huggingface/transformers.git
!cp transformers/examples/pytorch/language-modeling/run_clm.py /content/transformers/examples/pytorch/language-modeling/

Cloning into 'transformers'...
remote: Enumerating objects: 142383, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 142383 (delta 91), reused 142 (delta 66), pack-reused 142186
Receiving objects: 100% (142383/142383), 141.18 MiB | 15.61 MiB/s, done.
Resolving deltas: 100% (106475/106475), done.
cp: 'transformers/examples/pytorch/language-modeling/run_clm.py' and '/content/transformers/examples/pytorch/language-modeling/run_clm.py' are the same file


In [29]:
# Configuración del entrenamiento
model_checkpoint = "gpt2" # Modelo base
output_dir = "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ANXIETY" # Carpeta donde se guardarán los modelos y los logs de entrenamiento
num_train_epochs = 3 # Número de épocas de entrenamiento
learning_rate = 5e-5 # Tasa de aprendizaje
per_device_train_batch_size = 2 # Tamaño del lote de entrenamiento por dispositivo
per_device_eval_batch_size = 2 # Tamaño del lote de evaluación por dispositivo
warmup_steps = 500 # Número de pasos de calentamiento
logging_dir = "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/logs/generative/ANXIETY" # Carpeta donde se guardarán los registros de entrenamiento
logging_steps = 500 # Frecuencia de los registros de entrenamiento
save_steps = 500 # Frecuencia de guardar el modelo

In [30]:
cmd = f"python /content/transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path {model_checkpoint} \
    --train_file {train_file_path} \
    --validation_file {val_file_path} \
    --do_train \
    --do_eval \
    --output_dir {output_dir} \
    --num_train_epochs {num_train_epochs} \
    --learning_rate {learning_rate} \
    --per_device_train_batch_size {per_device_train_batch_size} \
    --per_device_eval_batch_size {per_device_eval_batch_size} \
    --warmup_steps {warmup_steps} \
    --logging_dir {logging_dir} \
    --logging_steps {logging_steps} \
    --save_steps {save_steps} \
    --fp16 \
    --overwrite_output_dir \
    --report_to tensorboard"

In [31]:
!pip install -r /content/transformers/examples/pytorch/language-modeling/requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.5 MB/s eta 0:00:00


In [32]:
!pip install git+https://github.com/huggingface/transformers --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00


In [33]:
# Se instala el módulo "evaluate" que no está instalado en el entorno en el que se está ejecutando el script
!pip install evaluate --quiet

In [34]:
%run /content/transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path gpt2 \
    --train_file "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ANXIETY/split/train.txt" \
    --validation_file "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ANXIETY/split/validation.txt"\
    --do_train \
    --do_eval \
    --output_dir "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ANXIETY" \
    --num_train_epochs 3 \
    --learning_rate 5e-5 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --warmup_steps 500 \
    --logging_dir "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/logs/generative/ANXIETY" \
    --logging_steps 500 \
    --save_steps 500 \
    --fp16 \
    --overwrite_output_dir \
    --report_to tensorboard

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=

INFO:datasets.download.download_manager:Downloading took 0.0 min
INFO:datasets.download.download_manager:Checksum Computation took 0.0 min


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:datasets.builder:Generating train split


Generating train split: 0 examples [00:00, ? examples/s]

...

[INFO|trainer.py:2904] 2023-05-23 14:15:15,553 >> Saving model checkpoint to /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ANXIETY/checkpoint-500
[INFO|configuration_utils.py:458] 2023-05-23 14:15:16,264 >> Configuration saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ANXIETY/checkpoint-500/config.json
[INFO|configuration_utils.py:364] 2023-05-23 14:15:16,538 >> Configuration saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ANXIETY/checkpoint-500/generation_config.json
[INFO|modeling_utils.py:1836] 2023-05-23 14:15:27,352 >> Model weights saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ANXIETY/checkpoint-500/pytorch_model.bin
[INFO|tokenization_utils_base.py:2181] 2023-05-23 14:15:27,744 >> tokenizer config file saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generativ

***** train metrics *****
  epoch                    =        3.0
  total_flos               =  3705690GF
  train_loss               =     3.3094
  train_runtime            = 0:28:09.03
  train_samples            =       2538
  train_samples_per_second =      4.508
  train_steps_per_second   =      2.254


INFO:__main__:*** Evaluate ***
[INFO|trainer.py:3165] 2023-05-23 14:40:25,908 >> ***** Running Evaluation *****
[INFO|trainer.py:3167] 2023-05-23 14:40:25,909 >>   Num examples = 305
[INFO|trainer.py:3170] 2023-05-23 14:40:25,910 >>   Batch size = 2


***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =      0.345
  eval_loss               =     3.2752
  eval_runtime            = 0:00:21.02
  eval_samples            =        305
  eval_samples_per_second =     14.503
  eval_steps_per_second   =      7.275
  perplexity              =    26.4488


[INFO|modelcard.py:451] 2023-05-23 14:40:47,600 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.345037898818967}]}


# Generamos una muestra de texto a partir de una semilla

In [35]:
from transformers import pipeline

In [36]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [37]:
!pip install xformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [38]:
model_path = r"/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ANXIETY/checkpoint-3500"

In [39]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

[INFO|tokenization_utils_base.py:1808] 2023-05-23 14:51:20,395 >> loading file vocab.json
[INFO|tokenization_utils_base.py:1808] 2023-05-23 14:51:20,401 >> loading file merges.txt
[INFO|tokenization_utils_base.py:1808] 2023-05-23 14:51:20,403 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:1808] 2023-05-23 14:51:20,403 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:1808] 2023-05-23 14:51:20,405 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:1808] 2023-05-23 14:51:20,406 >> loading file tokenizer_config.json
[INFO|configuration_utils.py:667] 2023-05-23 14:51:20,538 >> loading configuration file /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ANXIETY/checkpoint-3500/config.json
[INFO|configuration_utils.py:725] 2023-05-23 14:51:20,542 >> Model config GPT2Config {
  "_name_or_path": "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ANXIETY/checkpoint-3

In [40]:
# Definir la configuración del generador de texto
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [41]:
# Definir la semilla para la generación de texto
prompt = "do you have anxiety?"

In [42]:
# Generar texto a partir de la semilla
text = generator(prompt, max_length=150, do_sample=True, temperature=0.7)
print(text[0]['generated_text'])

[INFO|configuration_utils.py:577] 2023-05-23 14:52:24,585 >> Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.30.0.dev0"
}

[WARNING|utils.py:1282] 2023-05-23 14:52:24,587 >> Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


do you have anxiety? I mean, Ive had anxiety before, in the past, but this one is different. It started at work, and I had a lot of conversations with coworkers and coworkers and just basically they all just said they just want to talk to me but I just feel like Im just so anxious that I dont want to talk. I just feel like my heart is beating so fast, I feel like I might not be able to breathe, I feel like Im having a seizure, I dont know how to describe it. Im just so frustrated when I try to talk to people. I just feel this way to the point of me not being able to talk. I dont know how to explain it.
My boyfriend is a great
